## POSTPROECESSING

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !pip install leidenalg==0.7.0

In [3]:
%matplotlib inline
#%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

import torch
import numpy
import skimage.color
import torch.nn.functional as F
from MODULES.graph_clustering import *
import time
from typing import List, Tuple

## Helper function

In [4]:
def do_resolution_sweep(g, res_range, cpm_or_modularity, each_cc_separately):
    video_list, title_list, ncell_list, iou_list, resolution_list, mi_list, sizes_list = [], [], [], [], [], [], []
    for r in res_range:
        partition = g.find_partition_leiden(resolution=r, 
                                            cpm_or_modularity=cpm_or_modularity,
                                            each_cc_separately=each_cc_separately,
                                            min_size=10)
        concordance = partition.concordance_with_partition(other_partition=g.partition_sample_segmask)
        
        mi_list.append(concordance.mutual_information)
        iou_list.append(concordance.iou)
        seg_mask = g.partition_2_mask(partition)
        video_list.append(seg_mask)
        resolution_list.append(r)
        ncell_list.append(len(partition.sizes)-1)
        sizes_list.append(partition.sizes.cpu().numpy())
        
    video = torch.stack(video_list, dim=0).cpu().numpy()

    mydict = {"video": video, 
              "ref_image": g.raw_image[0].cpu().numpy(), 
              "resolution_list": resolution_list, 
              "ncell_list": ncell_list,
              "mi_list": mi_list,
              "iou_list": iou_list,
              "sizes_list": sizes_list}
    return mydict

In [5]:
def show_video(dictionary: dict,
        figsize: Tuple[int, int] = (8, 4),
        interval: int = 50,
        nbins=10,
        hist_range=(0,200)):
    """
    :param video: an ndarray with shape (n_frames, height, width)
    :param title: an list of lenght (n_frames)    
    """
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=figsize)
    ax_video = axs[0,0]
    ax_image = axs[0,1]
    ax_graph = axs[1,0]
    ax_hist = axs[1,1]
    
    # image
    ax_video.axis('off')
    im = ax_video.imshow(skimage.color.label2rgb(dictionary["video"][0, ...], bg_label=0), aspect='equal')
    ax_video.set_title("title")

    # reference image
    ax_image.axis('off')
    _ = ax_image.imshow(dictionary["ref_image"], cmap='gray', aspect='equal')
    
    # graph
    color = 'tab:red'
    _ = ax_graph.plot(dictionary["resolution_list"], dictionary["ncell_list"], '.--', label="n_cell", color=color)
    ax_graph.set_xlabel("resolution")
    ax_graph.set_ylabel('n_cell', color=color)
    ax_graph.tick_params(axis='y', labelcolor=color)
    ax_graph.legend()
    ax_graph.grid()

    ax_graph_2 = ax_graph.twinx()  # instantiate a second axes that shares the same x-axis
    color = 'tab:green'
    _ = ax_graph_2.plot(dictionary["resolution_list"], dictionary["iou_list"], '-', label="iou", color=color)
    ax_graph_2.set_ylabel('iou', color=color)
    ax_graph_2.tick_params(axis='y', labelcolor=color)
    ax_graph_2.legend()
    
    # hist
    _ = ax_hist.hist(dictionary["sizes_list"][0][1:], range=hist_range, density=False, bins=nbins)
    ax_hist.set_title("title")
        
    plt.tight_layout()
    plt.close()
    
    def init():
        im.set_data(dictionary["video"][0, ...])
        ax_video.set_title("title")
        ax_hist.hist(dictionary["sizes_list"][0][1:], range=hist_range, density=False, bins=nbins)
        
    def animate(i):
        title = 'frame={0:3d} res={1:.3f} ncell={2:2d}'.format(i,dictionary["resolution_list"][i],
                                                               dictionary["ncell_list"][i])
        ax_video.imshow(skimage.color.label2rgb(dictionary["video"][i, ...], bg_label=0))
        ax_video.set_title(title)
        ax_hist.clear()
        ax_hist.hist(dictionary["sizes_list"][i][1:], range=hist_range, density=False, bins=nbins)
        ax_hist.set_title("PDF cell size")
        
    anim = animation.FuncAnimation(
        fig, animate, init_func=init, frames=dictionary["video"].shape[0],
        interval=interval)
    
    return HTML(anim.to_html5_video())

# Import files

In [7]:
#tiling_file = "/home/jupyter/REPOS/spacetx-research/NEW_ARCHIVE/merfish_aug_5_v1/tiling_aug7_radius5.pt"
#tiling_file = "/Users/ldalessi/DAPI_unsupervised/spacetx-research/merfish_aug_5_v1/tiling_aug7_radius5.pt"
tiling_file = "/home/jupyter/REPOS/spacetx-research/NEW_ARCHIVE/Visium_aug_7_v1/tiling.pt"

# Load and plot results without normalizing the graph

In [16]:
tiling_original = torch.load(tiling_file, map_location=torch.device('cpu'))
#tiling_original = torch.load(tiling_file) #, map_location=torch.device('cpu'))

In [17]:
tiling_original.similarity

SparseSimilarity(csr_matrix=<4000000x4000000 sparse matrix of type '<class 'numpy.float64'>'
	with 29336228 stored elements in Compressed Sparse Row format>, index_matrix=tensor([[      0,       1,       2,  ...,    1997,    1998,    1999],
        [   2000,    2001,    2002,  ...,    3997,    3998,    3999],
        [   4000,    4001,    4002,  ...,    5997,    5998,    5999],
        ...,
        [3994000, 3994001, 3994002,  ..., 3995997, 3995998, 3995999],
        [3996000, 3996001, 3996002,  ..., 3997997, 3997998, 3997999],
        [3998000, 3998001, 3998002,  ..., 3999997, 3999998, 3999999]]), type='sparse')

In [18]:
g = GraphSegmentation(tiling_original, min_fg_prob=0.1, min_edge_weight=0.1, normalize_graph_edges=False)

KeyboardInterrupt: 

In [ ]:
g.suggest_resolution_parameter(window: Optional[tuple] = None,
                                         min_size: Optional[float] = 10,
                                         max_size: Optional[float] = None,
                                         cpm_or_modularity: str = "modularity",
                                         each_cc_separately: bool = False,
                                         show_graph: bool = True,
                                         figsize: tuple = (12, 12),
                                         fontsize: int = 20,
                                         sweep_range: Optional[numpy.ndarray] = None) -> Suggestion:
            """ This function select the resolution parameter which gives the hig

In [9]:
#sweep_CPM_radius_10_NO_norm = do_resolution_sweep(g, 
#                                          res_range=numpy.arange(0.1,10.0,0.5), 
#                                          each_cc_separately=False,
#                                          cpm_or_modularity="cpm")
#

sweep_MOD_radius_10_NO_norm = do_resolution_sweep(g, 
                                          res_range=numpy.arange(0.1,6.0,0.1), 
                                          each_cc_separately=False,
                                          cpm_or_modularity="modularity")

g = GraphSegmentation(tiling_original, min_fg_prob=0.1, min_edge_weight=0.1, normalize_graph_edges=True)

#sweep_CPM_radius_10_YES_norm = do_resolution_sweep(g, 
#                                          res_range=numpy.arange(0.1,10.0,0.5), 
#                                          each_cc_separately=False,
#                                          cpm_or_modularity="cpm")


sweep_MOD_radius_10_YES_norm = do_resolution_sweep(g, 
                                          res_range=numpy.arange(0.1,6.0,0.1), 
                                          each_cc_separately=False,
                                          cpm_or_modularity="modularity")

In [10]:
sweep_MOD_radius_10_YES_norm.keys()

dict_keys(['video', 'ref_image', 'resolution_list', 'ncell_list', 'mi_list', 'iou_list', 'sizes_list'])

In [11]:
sweep_MOD_radius_10_YES_norm["resolution_list"]

[0.1,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6,
 0.7000000000000001,
 0.8,
 0.9,
 1.0,
 1.1,
 1.2000000000000002,
 1.3000000000000003,
 1.4000000000000001,
 1.5000000000000002,
 1.6,
 1.7000000000000002,
 1.8000000000000003,
 1.9000000000000001,
 2.0,
 2.1,
 2.2,
 2.3000000000000003,
 2.4000000000000004,
 2.5000000000000004,
 2.6,
 2.7,
 2.8000000000000003,
 2.9000000000000004,
 3.0000000000000004,
 3.1,
 3.2,
 3.3000000000000003,
 3.4000000000000004,
 3.5000000000000004,
 3.6,
 3.7,
 3.8000000000000003,
 3.9000000000000004,
 4.0,
 4.1,
 4.2,
 4.3,
 4.3999999999999995,
 4.5,
 4.6,
 4.7,
 4.8,
 4.9,
 5.0,
 5.1,
 5.2,
 5.3,
 5.4,
 5.5,
 5.6,
 5.7,
 5.8,
 5.9]

In [12]:
show_video(dictionary=sweep_MOD_radius_10_YES_norm, interval=500, nbins=50, hist_range=(0,600))

In [ ]:
#tiling = tiling_original.reduce_similarity_radius(new_radius = 5)
#g = GraphSegmentation(tiling, min_fg_prob=0.1, min_edge_weight=0.1, normalize_graph_edges=False)

In [ ]:
#tiling = tiling_original.reduce_similarity_radius(new_radius = 2)
#g = GraphSegmentation(tiling, min_fg_prob=0.1, min_edge_weight=0.1, normalize_graph_edges=False)

# Examine the results

In [ ]:
def plot_comparison(which_list,cpm_or_modularity):

    fig, axs = plt.subplots(ncols=2, figsize=(12,6))
    fig.suptitle(which_list+"  "+cpm_or_modularity, fontsize=16)
    
    ax0 = axs[0]
    ax1 = axs[1]
    ax0.set_title("YES_norm")
    ax1.set_title("NO_norm")
    if which_list == "ncell_list":
        ax0.set_ylim(15,25)
        ax1.set_ylim(15,25)

    if cpm_or_modularity == "cpm":
        ax0.plot(sweep_CPM_radius_10_YES_norm["resolution_list"],sweep_CPM_radius_10_YES_norm[which_list], 'x--', label="CPM 10",color="blue")
        ax0.plot(sweep_CPM_radius_5_YES_norm["resolution_list"],sweep_CPM_radius_5_YES_norm[which_list], 'x--', label="CPM 5",color="red")
        ax0.plot(sweep_CPM_radius_2_YES_norm["resolution_list"],sweep_CPM_radius_2_YES_norm[which_list], 'x--', label="CPM 2",color="green")
        
        ax1.plot(sweep_CPM_radius_10_NO_norm["resolution_list"],sweep_CPM_radius_10_NO_norm[which_list], 'x--', label="CPM 10",color="blue")
        ax1.plot(sweep_CPM_radius_5_NO_norm["resolution_list"],sweep_CPM_radius_5_NO_norm[which_list], 'x--', label="CPM 5",color="red")
        ax1.plot(sweep_CPM_radius_2_NO_norm["resolution_list"],sweep_CPM_radius_2_NO_norm[which_list], 'x--', label="CPM 2",color="green")
    else:
        ax0.plot(sweep_MOD_radius_10_YES_norm["resolution_list"],sweep_MOD_radius_10_YES_norm[which_list], 'o-', label="MOD 10",color="blue")
        ax0.plot(sweep_MOD_radius_5_YES_norm["resolution_list"],sweep_MOD_radius_5_YES_norm[which_list], 'o-', label="MOD 5",color="red")
        ax0.plot(sweep_MOD_radius_2_YES_norm["resolution_list"],sweep_MOD_radius_2_YES_norm[which_list], 'o-', label="MOD 2",color="green")
        
        ax1.plot(sweep_MOD_radius_10_NO_norm["resolution_list"],sweep_MOD_radius_10_NO_norm[which_list], 'o-', label="MOD 10",color="blue")
        ax1.plot(sweep_MOD_radius_5_NO_norm["resolution_list"],sweep_MOD_radius_5_NO_norm[which_list], 'o-', label="MOD 5",color="red")
        ax1.plot(sweep_MOD_radius_2_NO_norm["resolution_list"],sweep_MOD_radius_2_NO_norm[which_list], 'o-', label="MOD 2",color="green")
       
    ax0.grid()
    ax0.legend()
    ax0.set_xlabel('resolution parameter')
    
    ax1.grid()
    ax1.legend()
    ax1.set_xlabel('resolution parameter')

In [ ]:
plot_comparison("iou_list","modularity")

In [ ]:
plot_comparison("ncell_list","modularity")

In [ ]:
plot_comparison("ncell_list","cpm")

In [ ]:
plot_comparison("iou_list","cpm")

# SHOW A VIDEO WITH AND WITHOUT EDGE NORMALIZATION

In [ ]:
#show_video(dictionary=sweep_MOD_radius_2_NO_norm, interval=500, nbins=50, hist_range=(0,300))

In [ ]:
show_video(dictionary=sweep_MOD_radius_5_NO_norm, interval=500, nbins=50, hist_range=(0,600))

In [ ]:
#show_video(dictionary=sweep_MOD_radius_10_NO_norm, interval=500, nbins=50, hist_range=(0,200))

In [ ]:
#show_video(dictionary=sweep_CPM_radius_2, interval=500, nbins=50, hist_range=(0,200))

In [ ]:
#show_video(dictionary=sweep_CPM_radius_5, interval=500, nbins=50, hist_range=(0,200))

In [ ]:
#show_video(dictionary=sweep_CPM_radius_10, interval=500, nbins=50, hist_range=(0,200))

# MANUAL ANALYSIS

In [ ]:
tiling_original = torch.load(mask_file_dot, map_location=torch.device('cpu'))
tiling = tiling_original.reduce_similarity_radius(new_radius = 5)
g = GraphSegmentation(tiling, min_fg_prob=0.1, min_edge_weight=0.1)

In [ ]:
# graph summary

g.graph.summary()

In [ ]:
_ = g.plot_partition()

In [ ]:
#_ = g.plot_partition(window=(40,0,100,40))
_ = g.plot_partition(window=(0,40,100,1000))

In [ ]:
suggestion = g.suggest_resolution_parameter(window=(0,40,100,1000))
print(suggestion.best_resolution)

In [ ]:
suggestion.show_best(figsize=(12,12))